In [119]:
# Getting the input data
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy import stats 
import shutil


In [120]:
os.chdir('G:\Wyatt_Olea\Compressive\Wood_3')

# Reading all the data in csv format to the data frame

output = pd.read_csv("output.csv",skiprows = 2,names = ['Time','Force','Disp'])
output = output.apply(pd.to_numeric, errors ='coerce')
output = output.dropna(axis = 0)

eystrain = pd.read_csv("ey_strain.csv",skiprows = 3,names =['Stage','ey_strain'])
eystrain = eystrain.apply(pd.to_numeric, errors='coerce')
eystrain = eystrain.dropna(axis = 0)


# Dimensions = pd.read_table("Dimensions.txt")

print('Input_length:')
print(output.head(5))
# print(Dimensions)


Input_length:
       Time  Force  Disp
0  0.480798     -1   0.0
1  0.648533     -1   0.0
2  0.880576     -2   0.0
3  1.090825     -2   0.0
4  1.237819     -3   0.0


In [121]:
#Calculating Stress from the output data 
# Area = int(Dimensions["Width"]*Dimensions["Thickness"])
Area = 7.784


# Changing strain from percentage to mm/mm format
eystrain["ey_strain"] = eystrain["ey_strain"]


# Calculating Stress from force 
output["stress"] = -1*(output["Force"]/Area) 

# Assembling all the data to a data frame
data = pd.concat(
    [output['Time'], output['stress'], eystrain['Stage'], eystrain['ey_strain']], 
    axis = 1, keys= ['time','stress','stage','strain'])

print(Area)
print(data['time'].max())
data


7.784
181.6744533


,time,stress,stage,strain
0,0.480798,0.128469,0.0,0.000000
1,0.648533,0.128469,1.0,0.005720
2,0.880576,0.256937,2.0,0.019696
3,1.090825,0.256937,3.0,0.019874
4,1.237819,0.385406,4.0,0.043672
5,1.477582,0.385406,5.0,0.045199
6,1.651315,0.513875,6.0,0.061141
7,1.849305,0.513875,7.0,0.077672
8,2.077815,0.642343,8.0,0.070424
9,2.259108,0.770812,9.0,0.086972


In [122]:
#Filtering stress

#Filtering out the Stress data corresponding to each Strain Stage

Stress = []
time = np.array(data['time'])
stage = np.array(data['stage'].dropna())
time_sorted = []


# Finding closest time value of stress corresponding to each strain stage data
for i in range(0, len(stage)):
    time_sorted.append(min(range(len(time)), key=lambda j: abs(time[j]-stage[i])))

for i in range (0, len(time_sorted)):
        Stress.append(data['stress'][time_sorted[i]]) 

Strain = np.array(data["strain"].dropna())
Stress = np.array(Stress)


print(len(Strain))
print(len(Stress))

Data = pd.DataFrame({'Strain':Strain,'Stress':Stress},columns = ['Strain','Stress'])
Data.to_csv('Data.csv')




182
182


# Finding Mechanical Properties 

In [123]:
# Function to calculate Mechanical Properties

def Mechanical_Properties(stress, strain, split_stress):

    #Calculating Young's Modulus from raw data ey_strain

    #Filtering out the linear potion of the ey_strain and stress data to find the slope

    linear_strain = []
    linear_stress = []

    for i in range(0, len(stress)):
        if stress[i] < split_stress:
            linear_strain.append(strain[i])
            linear_stress.append(stress[i])
        
    linear_strain = np.array(linear_strain)
    linear_stress = np.array(linear_stress)
            
    # Filtering out the linear potion of the ey_strain and stress data to fit a polynomial

    nonlinear_strain =[]
    nonlinear_stress =[]

    for i in range(0,len(stress)):
        if stress[i] > split_stress:
            nonlinear_strain.append(strain[i])
            nonlinear_stress.append(stress[i])
            
    #Fitting a polynomial curve to the nonlinear portion of stress-strain curve
    p = np.polyfit(nonlinear_strain,nonlinear_stress,12)
    p1d = np.poly1d(p)
    
    #Calculating Young's Modulus
    E,c,r,_,_ = stats.linregress(linear_strain,linear_stress)  
    
    #Calculating R^2 value
    K = len(linear_stress)
    r_squared =  1 - r /(K * linear_stress.var())
    r_squared = round(r_squared,5)

    #Calculating coefficient of variation
    V = 100*np.sqrt(((1/r_squared)-1)/(K-2))

    #Finding Ultimate Stress and Strain
    ucs = np.amax(stress)
    ucstrain = np.amax(strain)
    
    #To find yield stress from initially filtered data of Engineering strain vs Engineering stress
    #This works by finding intersection point of the straight line and polynomial

    x = np.linspace(0.002,0.05,1000)

    for i in range(0,len(x)):
        if (E*(x[i] -0.002)) > (p1d(x[i])):
            yieldstrain = x[i]
            yieldstress = p1d(x[i])
            break 
    
        
    #Caculating ductility from filtered data ey_strain
        
    ductility = (ucstrain - yieldstrain)*100
    ductility = round(ductility,2)

    return(round(yieldstress,2),round(ucs,2),int(E),round(ductility,2),round(r_squared,5),K,round(V,5))
    
print("done")


done


In [124]:
#Mechanical properties in different processing conditions

split_stress = 20   # Passing uniaxial strain, uniaxial stress
print(split_stress)

[y,ucs,E,d,R2,K,V] = Mechanical_Properties(Stress,Strain,split_stress)


dir = 'Results'
if not os.path.exists(dir):
    os.makedirs(dir)
else:
    shutil.rmtree(dir)           #removes all the subdirectories!
    os.makedirs(dir)

os.chdir('Results')



Data = pd.DataFrame({'Strain':Strain,'Stress':Stress},columns = ['Strain','Stress'])

Results = pd.DataFrame({'Result Type':['Uniaxial'],'Yield Strength(Mpa)':[y],'Ultimate Strength(Mpa)':[ucs],
                        'Youngs Modulus(Gpa)':[E/1000],'Ductility':[d],'R^2':[R2],'Data Points':[K],'CoV(%)':[V]},
                        columns = ['Result Type','Yield Strength(Mpa)','Ultimate Strength(Mpa)','Youngs Modulus(Gpa)',
                                   'Ductility','R^2','Data Points','CoV(%)'])

Results.to_csv('Results.csv')
Data.to_csv('Data.csv')

print(Results)


20


UnboundLocalError: local variable 'yieldstrain' referenced before assignment

In [ ]:
# Plotting 

# To draw straight line in graph

b = np.linspace(0,0.002,1000)
g = np.linspace(0.002,0.002,2000)

a = E*b
c = E*(g - 0.002)

#plt.clf()

plt.figure(figsize = (20,15))
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.plot(Strain, Stress, 'k-')
plt.plot(b,a,'-')
plt.plot(g,c,'-')
plt.xlabel('Engineering Strain (mm/mm)', fontsize = 20)
plt.ylabel('Engineering Stress (Mpa)', fontsize = 20)
plt.xlim([0,Strain.max()+0.001])
plt.ylim([0,Stress.max()+5])
plt.savefig('Eng_Strain_Stress.tiff')


plt.show()
